In [1]:
import pandas as pd
import numpy as np
import os, sys
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
df= pd.read_excel("data/default of credit card clients.xls", skiprows=[0])
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [3]:
cat_features=['SEX', 'EDUCATION','MARRIAGE']
num_features=['LIMIT_BAL','AGE','PAY_0','PAY_2','PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 
            'BILL_AMT1', 'BILL_AMT2','BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
        'PAY_AMT1','PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']

       

In [3]:
df["SEX"].unique()

array([2, 1], dtype=int64)

In [4]:
df["EDUCATION"].unique()

array([2, 1, 3, 5, 4, 6, 0], dtype=int64)

In [5]:
df["MARRIAGE"].unique()

array([1, 2, 3, 0], dtype=int64)

In [6]:
df['PAY_0'].unique()

array([ 2, -1,  0, -2,  1,  3,  4,  8,  7,  5,  6], dtype=int64)

In [4]:
## importing sklearn libraries 
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [5]:
class featuregenerator(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.bill_amt1_ind=8
        self.bill_amt2_ind=9
        self.bill_amt3_ind=10
        self.bill_amt4_ind=11
        self.bill_amt5_ind=12
        self.bill_amt6_ind=13
        self.pay_amt1_ind=14
        self.pay_amt2_ind=15
        self.pay_amt3_ind=16
        self.pay_amt4_ind=17
        self.pay_amt5_ind=18
        self.pay_amt6_ind=19

    def fit(self, x,y=None):
        return self
    
    def transform(self,x, y=None):
        slack_sept= x[:,self.bill_amt1_ind]- x[:, self.pay_amt1_ind]
        slack_aug=  x[:,self.bill_amt2_ind]- x[:, self.pay_amt2_ind]
        slack_july= x[:,self.bill_amt3_ind]- x[:, self.pay_amt3_ind]
        slack_jun=  x[:,self.bill_amt4_ind]- x[:, self.pay_amt4_ind]
        slack_may=  x[:,self.bill_amt5_ind]- x[:, self.pay_amt5_ind]
        slack_apr=  x[:,self.bill_amt6_ind]- x[:, self.pay_amt6_ind]

        x_new= np.c_[x[:,0:8],slack_sept, slack_aug, slack_july,slack_jun, slack_may,slack_apr]

        return x_new

In [6]:
num_pipeline=Pipeline(steps=[("Imputer",SimpleImputer(strategy='median')),
                             ('featuregeneration', featuregenerator()),
                             ("Scaling", StandardScaler())])

cat_pipeline= Pipeline(steps=[("Imputer",SimpleImputer(strategy='most_frequent')), 
                               ("OnehotEncoding", OneHotEncoder(drop='first'))])

preprocessor=ColumnTransformer([('num_pipeline',num_pipeline, num_features),
                                      ("Cat_pipeline",cat_pipeline,cat_features)])

In [7]:
X= df.drop(columns=['default payment next month'])
Y= df['default payment next month']
x_train, x_test, y_train, y_test= train_test_split(X, Y, test_size=0.25)

In [8]:
preprocessor.fit(x_train)

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('Imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('featuregeneration',
                                                  featuregenerator()),
                                                 ('Scaling',
                                                  StandardScaler())]),
                                 ['LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3',
                                  'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1',
                                  'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4',
                                  'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
                                  'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4',
                                  'PAY_AMT5', 'PAY_AMT6']),
                                ('Cat_pipeline',
                                 Pipeline(steps=[('I

In [9]:
x_train_transformed= preprocessor.transform(x_train)
x_test_transformed=preprocessor.transform(x_test)

In [46]:
x_train.shape, x_train_transformed.shape

((22500, 24), (22500, 24))

In [55]:
rmse_train=[]
rmse_test=[]
model_name=[]
def train_model(model, x_df= x_train_transformed, y_df=y_train, x_test=x_test_transformed, y_test=y_test):
    model.fit(x_train_transformed,y_train)
    y_pred_train= model.predict(x_train_transformed)
    y_pred_test= model.predict(x_test_transformed)
    rmse_train.append(model.score(x_train_transformed,y_train))
    rmse_test.append(model.score(x_test_transformed,y_test))
    model_name.append(model)

In [56]:
train_model(LogisticRegression())

c:\Users\HP\.conda\envs\dfenv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [50]:
train_model(DecisionTreeClassifier())

In [51]:
train_model(RandomForestClassifier())

In [52]:
train_model(XGBClassifier())

In [53]:
train_model(SVC())

In [54]:
analysis_df= pd.DataFrame({"Model":model_name,"RMSETRAIN":rmse_train,"RMSETEST":rmse_test})
analysis_df

,Model,RMSETRAIN,RMSETEST
0,LogisticRegression(),0.813689,0.804667
1,DecisionTreeClassifier(),0.999333,0.727200
2,"(DecisionTreeClassifier(max_features='auto', r...",0.999289,0.807867
3,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.874578,0.808267
4,SVC(),0.826489,0.812400


In [55]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')

In [60]:
x_new= df.loc[0:5,['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6','default payment next month']]